In [59]:
import warnings
warnings.filterwarnings("ignore")

In [60]:
import pandas as pd 
df = pd.read_csv("/home/prasun/GitDemo/Stat_ML-Learning/Exercise19/dataset.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [61]:
smr = df.describe()
smr

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [62]:
for i in range(0,len(smr)-1):
    IQR = smr.iloc[6,i]-smr.iloc[4,i]
    min = (smr.iloc[4,i]-(1.5*IQR))
    max = (smr.iloc[6,i]+(1.5*IQR))
    print(f"({min}, {max})")



(27.5, 79.5)
(90.0, 170.0)
(32.625, 407.625)
(0.0, 0.0)
(66.0, 210.0)
(-2.25, 3.75)
(-1.5, 2.5)


In [63]:
smr.loc['25%':'75%','Age']

25%    47.0
50%    54.0
75%    60.0
Name: Age, dtype: float64

In [104]:
# Z_Score
from scipy.stats import zscore
num = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
df_zscore  =df[num].apply(zscore)
dfb = [(df_zscore< 3) & (df_zscore> -3)]

for i in num:
    dff = df.loc[dfb[0][i]]
dff.shape

(911, 12)

In [65]:
df.shape

(918, 12)

In [78]:
df.loc[(df.Cholesterol < 407.625) & (df.Cholesterol> 32.625)].shape

(735, 12)

In [107]:
X_num = df[['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak',]]
X_cat = df.drop(['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak','HeartDisease'], axis= "columns")

num = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
cat = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

y = df[['HeartDisease']]
y_zc = dff[['HeartDisease']]
X = df.drop(['HeartDisease'], axis= "columns")

print(f" Numeric: {X_num.shape}, Categorical: {X_cat.shape}")

 Numeric: (918, 6), Categorical: (918, 5)


In [105]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


label = OneHotEncoder()
scale = MinMaxScaler()
pca1 = PCA(n_components= 2)
pipe = Pipeline([("scale",scale),
                 ("pca",pca1)])
final = ColumnTransformer(transformers= [
    ("num", pipe, num),
    ("cat",label, cat)
], remainder= "drop")

X_final = final.fit_transform(df)
X_zc =  final.fit_transform(dff)

In [69]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np

log = LogisticRegression()
sv = SVC()
ran = RandomForestClassifier()

sc1 = np.mean(cross_val_score(log,X_final,y,cv = 10) )
sc2 = np.mean(cross_val_score(sv,X_final,y,cv = 10))
sc3 = np.mean(cross_val_score(ran,X_final,y,cv = 10))

print(f"Logistic: {sc1},\nSupport Vector: {sc2}\nRandom Forest: {sc3}")


Logistic: 0.845102723363593,
Support Vector: 0.837494027711419
Random Forest: 0.8223363592928811


In [108]:
# Z Score Apllied
sc1 = np.mean(cross_val_score(log,X_zc,y_zc,cv = 10) )
sc2 = np.mean(cross_val_score(sv,X_zc,y_zc,cv = 10))
sc3 = np.mean(cross_val_score(ran,X_zc,y_zc,cv = 10))

print(f"Logistic: {sc1},\nSupport Vector: {sc2}\nRandom Forest: {sc3}")

Logistic: 0.8496297181079789,
Support Vector: 0.8385809842331582
Random Forest: 0.8101170568561873


In [70]:
from sklearn.model_selection import GridSearchCV
gcv = GridSearchCV(log,param_grid={'penalty':['l1', 'l2', 'elasticnet'],
                                   "C": [1,5,10,20],
                                   "solver": ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky'],
                                   "multi_class": ['auto', 'ovr', 'multinomial']},
                                   cv= 10)
gcv.fit(X_final,y)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [1, 5, 10, 20],
                         'multi_class': ['auto', 'ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky']})

In [109]:
# Z Score Applied
from sklearn.model_selection import GridSearchCV
gcv = GridSearchCV(log,param_grid={'penalty':['l1', 'l2', 'elasticnet'],
                                   "C": [1,5,10,20],
                                   "solver": ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky'],
                                   "multi_class": ['auto', 'ovr', 'multinomial']},
                                   cv= 10)
gcv.fit(X_zc,y_zc)

GridSearchCV(cv=10,
             estimator=LogisticRegression(C=1, multi_class='auto', penalty='l1',
                                          solver='liblinear'),
             param_grid={'C': [1, 5, 10, 20],
                         'multi_class': ['auto', 'ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky']})

In [71]:
log = LogisticRegression(C=1, multi_class='auto', penalty='l1', solver='liblinear')
score = np.mean(cross_val_score(log,X_final,y,cv = 10))
print(f"Final Score: {score}")

Final Score: 0.8505494505494505


In [111]:
# Z Score apllied
log = LogisticRegression(C=1, multi_class='multinomial', solver='newton-cg')
score = np.mean(cross_val_score(log,X_zc,y_zc,cv = 10))
print(f"Final Score: {score}")

Final Score: 0.85072861920688
